# Finetuning Yolov11m

In [13]:
from ultralytics import YOLO
import os

# load the pretrained medium model
model = YOLO("yolo11l.pt")

# Use absolute path to YAML
yaml_path = "/home/joey/Projects/chatbrain/backend/vision_model/chat_dataset.yaml"

# Verify paths
assert os.path.exists(yaml_path), "YAML file not found!"
assert os.path.exists(os.path.join(os.path.dirname(yaml_path), "dataset/train/images")), "Train images missing!"
assert os.path.exists(os.path.join(os.path.dirname(yaml_path), "dataset/val/images")), "Val images missing!"

results = model.train(
    data=yaml_path,
    epochs=100,
    time=0.05, # maxium training time in hours
    patience=10, # early stopping patience
    batch=-1, # auto batch size
    imgsz=640, # image size
    multi_scale=True,  # Vary imgsz ±50% during training
    cache=True, # cache images for faster training
)

Ultralytics 8.3.67 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1060 3GB, 3004MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=/home/joey/Projects/chatbrain/backend/vision_model/chat_dataset.yaml, epochs=100, time=0.05, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sa

RuntimeError: Dataset '/home/joey/Projects/chatbrain/backend/vision_model/chat_dataset.yaml' error ❌ 
Dataset '/home/joey/Projects/chatbrain/backend/vision_model/chat_dataset.yaml' images not found ⚠️, missing path '/home/joey/Projects/datasets/dataset/val/images'
Note dataset download directory is '/home/joey/Projects/datasets'. You can update this in '/home/joey/.config/Ultralytics/settings.json'

#### Sort detected boxes vertically to get message order

In [6]:
def process_results(results):
    boxes = []
    for box in results[0].boxes:
        cls = int(box.cls)
        coords = box.xyxy[0].tolist()  # [ax, ay, bx, by]
        boxes.append((cls, coords))
    
    # Sort non-contact boxes by Y-coordinate (ay)
    contact_boxes = [b for b in boxes if b[0] == 0]
    message_boxes = sorted(
        [b for b in boxes if b[0] != 0],
        key=lambda x: x[1][1]  # Sort by ay (top Y-coordinate)
    )
    
    # Assign 'num' (0 for contacts, 1,2,3... for messages)
    final = []
    for cls, (ax, ay, bx, by) in contact_boxes:
        final.append([ax, ay, bx, by, 0, 0])  # Type 0, num=0
    
    for i, (cls, (ax, ay, bx, by)) in enumerate(message_boxes, 1):
        final.append([ax, ay, bx, by, cls, i])  # Type 1/2, num=i
    
    return final